In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd

In [2]:
cd 'drive'/'MyDrive'/'Dacon'/'LDI'

/content/drive/MyDrive/Dacon/LDI


In [3]:
test_err_data = pd.read_csv('./dataset/test_err_data.csv',thousands = ',')
test_quality_data = pd.read_csv('./dataset/test_quality_data.csv',thousands = ',')

In [4]:
df_ted = test_err_data.drop_duplicates()
df_tqd = test_quality_data.drop_duplicates()

In [5]:
df_ted.loc[(df_ted['errtype'] == 23) & (df_ted['errcode'] == 'connection timeout'),'errtype'] = '23_ct'
df_ted.loc[(df_ted['errtype'] == 23) & (df_ted['errcode'] == 'active'),'errtype'] = '23_as'
df_ted.loc[(df_ted['errtype'] == 23) & (df_ted['errcode'] == 'standby'),'errtype'] = '23_ac'
df_ted.loc[(df_ted['errtype'] == 31) & (df_ted['errcode'] == '0'),'errtype'] = '31_0'
df_ted.loc[(df_ted['errtype'] == 40) & (df_ted['errcode'] == '0'),'errtype'] = '40_0'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [6]:
df_tqd.describe()

,time,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
count,2.438290e+05,243829.000000,218101.000000,243818.000000,237889.000000,243829.0,243829.0,243805.000000,243829.000000,243829.000000,243829.000000,2.438290e+05,2.438290e+05,243829.000000,243829.000000
mean,2.020112e+13,37379.351874,6.411667,-0.026782,19.555902,0.0,0.0,198.772437,3.220290,28.152094,0.767427,1.429262e+02,1.671426e+03,-0.104897,0.038170
std,8.797857e+06,4242.786365,373.474737,21.194121,2095.919553,0.0,0.0,3275.166768,35.212068,307.311331,74.014589,1.215819e+04,2.836120e+04,0.349197,0.373594
min,2.020103e+13,30000.000000,-1.000000,-1.000000,-1.000000,0.0,0.0,-1.000000,-1.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,-1.000000,0.000000
25%,2.020111e+13,33783.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,3.000000e+00,0.000000,0.000000
50%,2.020112e+13,37166.000000,0.000000,0.000000,0.000000,0.0,0.0,2.000000,0.000000,0.000000,0.000000,0.000000e+00,1.200000e+01,0.000000,0.000000
75%,2.020112e+13,41064.000000,0.000000,0.000000,0.000000,0.0,0.0,8.000000,0.000000,0.000000,0.000000,0.000000e+00,1.100000e+02,0.000000,0.000000
max,2.020113e+13,44997.000000,93038.000000,10452.000000,636619.000000,0.0,0.0,156398.000000,600.000000,7200.000000,10452.000000,1.954316e+06,1.172849e+06,17.000000,19.000000


In [7]:
for i in range(0,13):
    qstr = 'quality_'+str(i)
    df_tqd.loc[df_tqd[qstr] != -1,qstr] = 0
    df_tqd.loc[df_tqd[qstr] == -1,qstr] = 1

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [8]:
df_ted = df_ted.drop('fwver',axis=1)
df_ted = df_ted.drop('model_nm',axis=1)
df_tqd = df_tqd.drop('fwver',axis=1)
df_ted = df_ted.drop('errcode',axis=1)

In [9]:
times = list(df_ted['time'])

In [10]:
times = list(df_ted['time'])

for i in range(len(times)):
    try:
        times[i] = int(str(times[i])[4:8])
    except ValueError:
        print(i)
    

df_ted['time'] = times

df_ted.head()

,user_id,time,errtype
0,30000,1101,31
1,30000,1101,33
2,30000,1101,15
3,30000,1101,22
4,30000,1101,11


In [11]:
print(len(df_ted['time'].unique()))
print(len(df_ted['errtype'].unique()))

39
46


In [12]:
df_err = df_ted[['user_id','errtype']].values
df_err

array([[30000, 31],
       [30000, 33],
       [30000, 15],
       ...,
       [44998, 15],
       [44998, 16],
       [44998, '31_0']], dtype=object)

In [13]:
from tqdm import tqdm

df_x = np.zeros((14999,60))
df_err = df_ted[['user_id','errtype']].values
for pi,err in tqdm(df_err):
    if err == '23_ct':
        err = 43
    elif err == '23_as':
        err = 44
    elif err == '23_ac':
        err = 45
    elif err == '31_0':
        err = 46
    elif err == '40_0':
        err = 47
    df_x[int(pi)-30000,int(err)-1] += 1

100%|██████████| 15527225/15527225 [00:34<00:00, 454015.96it/s]


In [14]:
df_tqd[df_tqd.user_id == 44997]

,time,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
747948,20201102214000,44997,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0
747952,20201102214000,44997,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0
747955,20201102214000,44997,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0
747960,20201125180000,44997,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0
747962,20201125180000,44997,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0
747967,20201125180000,44997,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0


In [15]:
df_tqd.describe()

,time,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
count,2.438290e+05,243829.000000,243829.000000,243829.000000,243829.000000,243829.0,243829.0,243829.000000,243829.000000,243829.0,243829.0,243829.0,243829.0,243829.000000,243829.0
mean,2.020112e+13,37379.351874,0.100853,0.114671,0.109368,0.0,0.0,0.114166,0.114511,0.0,0.0,0.0,0.0,0.114720,0.0
std,8.797857e+06,4242.786365,0.301135,0.318625,0.312101,0.0,0.0,0.318014,0.318431,0.0,0.0,0.0,0.0,0.318684,0.0
min,2.020103e+13,30000.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
25%,2.020111e+13,33783.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
50%,2.020112e+13,37166.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
75%,2.020112e+13,41064.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
max,2.020113e+13,44997.000000,1.000000,1.000000,1.000000,0.0,0.0,1.000000,1.000000,0.0,0.0,0.0,0.0,1.000000,0.0


In [16]:
for i in range(0,13):
    a = "quality_"+ str(i)
    df_qua = df_tqd[['user_id',a]].values
    for pi, qua in tqdm(df_qua):
        if qua == 1:
            df_x[int(pi)-30000,i + 47] += 1

100%|██████████| 243829/243829 [00:00<00:00, 651092.80it/s]


In [17]:
columns = []
for i in range(1,43):
    s = 'err_' + str(i)
    columns.append(s)
columns.append('23_ct')
columns.append('23_as')
columns.append('23_ac')
columns.append('31_0')
columns.append('40_0')
for i in range(13):
    s = 'quality_' + str(i)
    columns.append(s)
df = pd.DataFrame(df_x,columns=columns)
df.head()

,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_29,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,23_ct,23_as,23_ac,31_0,40_0,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,0.0,0.0,0.0,0.0,44.0,1.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,3.0,66.0,37.0,9.0,3.0,0.0,1.0,0.0,983.0,56.0,5.0,2.0,32.0,0.0,0.0,0.0,0.0,43.0,2.0,16.0,0.0,1.0,3.0,3.0,0.0,2.0,46.0,0.0,2.0,1107.0,141.0,4.0,40.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,41.0,2.0,1.0,0.0,0.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,0.0,0.0,0.0,60.0,1.0,2.0,0.0,40.0,0.0,0.0,0.0,0.0,108.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,14.0,57.0,1.0,2.0,26.0,7.0,6.0,122.0,56.0,3.0,3.0,3.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0
3,0.0,0.0,0.0,0.0,73.0,2.0,2.0,0.0,0.0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,5.0,2.0,2.0,0.0,0.0,65.0,0.0,4.0,7.0,3.0,1.0,1.0,0.0,0.0,19.0,8.0,0.0,1.0,1.0,0.0,69.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,10.0,12.0,1.0,10.0,79.0,67.0,6.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,14.0,1.0,1.0,0.0,0.0,164.0,152.0,11.0,5.0,1.0,1.0,1.0,1.0,0.0,24.0,90.0,0.0,0.0,5.0,3.0,168.0,24.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [18]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
df_s = ss.fit_transform(df)
df= pd.DataFrame(df_s,columns=df.columns)
df.describe()

,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_8,err_9,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_29,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,23_ct,23_as,23_ac,31_0,40_0,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
count,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,14999.0,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,1.499900e+04,14999.0,14999.0,1.499900e+04,1.499900e+04,14999.0,14999.0,14999.0,14999.0,1.499900e+04,14999.0
mean,-2.018520e-16,1.577815e-16,3.151365e-16,-4.269846e-16,5.848527e-16,2.940344e-17,2.282465e-16,1.156589e-15,1.699402e-17,-8.712348e-16,-3.532114e-16,1.437243e-16,1.547014e-17,6.465491e-17,-1.006854e-17,4.377531e-17,-2.359418e-16,4.120738e-16,-8.600490e-16,5.252704e-16,-3.856358e-16,-1.796257e-16,-1.487257e-15,-6.216933e-16,-1.294081e-15,-7.363721e-17,-8.795403e-18,-6.276278e-16,0.0,-1.428841e-16,-9.263063e-16,8.805951e-16,2.746944e-16,-2.259894e-16,5.709888e-17,-5.349263e-16,-5.673026e-16,-5.100020e-16,-1.124411e-15,2.020148e-16,-6.786469e-16,4.173681e-16,3.897344e-17,-1.831703e-16,-5.621267e-16,8.991878e-16,-1.668406e-16,-1.303489e-16,-5.768363e-17,4.256139e-16,0.0,0.0,8.726195e-17,1.168403e-16,0.0,0.0,0.0,0.0,5.077758e-18,0.0
std,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,0.0,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,0.0,0.0,1.000033e+00,1.000033e+00,0.0,0.0,0.0,0.0,1.000033e+00,0.0
min,-1.956875e-01,-3.126095e-02,-5.379082e-02,-2.421339e-02,-2.465166e-01,-1.042494e-01,-1.047077e-01,-4.734287e-02,-5.671497e-02,-2.284224e-02,-7.885530e-02,-4.098288e-01,-1.447602e-01,-5.434073e-02,-1.523081e+00,-1.303459e+00,-1.218395e-01,-2.028336e-01,-8.512677e-02,-1.901743e-01,-7.831786e-02,-2.123607e-01,-2.267505e-02,-8.135405e-02,-1.158500e-01,-1.552620e+00,-5.085978e-02,-4.938315e-02,0.0,-1.534718e-01,-1.442748e-01,-1.999311e-01,-1.000323e+00,-5.288296e-02,-7.533853e-02,-1.005008e+00,-1.000353e+00,-1.339658e-01,-8.672562e-02,-6.092053e-01,-3.907817e-01,-6.451818e-01,-1.660840e-01,-1.064470e-01,-8.232382e-02,-1.359429e-01,-5.951403e-01,-2.455327e-01,-2.608949e-01,-2.603369e-01,0.0,0.0,-2.620139e-01,-2.611658e-01,0.0,0.0,0.0,0.0,-2.607966e-01,0.0
25%,-1.956875e-01,-3.126095e-02,-5.379082e-02,-2.421339e-02,-2.465166e-01,-1.042494e-01,-1.047077e-01,-4.734287e-02,-5.671497e-02,-2.284224e-02,-3.100419e-02,-1.254870e-01,-1.447602e-01,-5.434073e-02,-6.502134e-01,-6.472763e-01,-1.218395e-01,-2.028336e-01,-8.512677e-02,-1.901743e-01,-7.831786e-02,-2.123607e-01,-2.267505e-02,-8.135405e-02,-1.158500e-01,-5.830224e-01,-5.085978e-02,-4.938315e-02,0.0,-1.534718e-01,-1.442748e-01,-1.999311e-01,-1.000323e+00,-5.288296e-02,-7.533853e-02,-1.005008e+00,-1.000353e+00,-1.339658e-01,-8.672562e-02,-6.092053e-01,-3.907817e-01,-6.451818e-01,-1.66084

In [19]:
df.drop(['err_29','quality_3','quality_4','err_9','err_10','err_16','23_ct','err_18','err_19','err_21','quality_11','quality_6','err_28','quality_2','quality_0','err_6','err_11','err_12','err_35','quality_10','err_17','err_36','quality_8','31_0','40_0'], axis = 1, inplace = True)

In [20]:
df.shape

(14999, 35)

In [22]:
df.head()

,err_1,err_2,err_3,err_4,err_5,err_7,err_8,err_13,err_14,err_15,err_20,err_22,err_23,err_24,err_25,err_26,err_27,err_30,err_31,err_32,err_33,err_34,err_37,err_38,err_39,err_40,err_41,err_42,23_as,23_ac,quality_1,quality_5,quality_7,quality_9,quality_12
0,-0.195688,-0.031261,-0.053791,-0.024213,0.023931,-0.076920,-0.047343,-0.144760,-0.045622,-0.562927,1.097021,1.435591,0.070512,0.062694,2.92463,-0.142296,-0.050860,-0.153472,-0.053083,-0.194506,0.347754,-0.052883,3.765437,-0.133966,0.040200,0.517091,-0.390782,-0.080476,0.891900,-0.047012,-0.260895,-0.262014,0.0,0.0,0.0
1,-0.195688,-0.031261,0.051418,0.005931,-0.228077,0.006443,-0.047343,0.676986,-0.036903,-0.752048,-0.190174,-0.212361,-0.022675,-0.081354,-0.11585,-0.494877,-0.046439,-0.153472,-0.144275,-0.199931,-1.000323,-0.052883,-1.000353,-0.133966,-0.086726,-0.609205,-0.390782,-0.645182,-0.106447,-0.082324,-0.120939,-0.120837,0.0,0.0,0.0
2,-0.195688,-0.031261,-0.053791,-0.024213,0.005492,-0.076920,-0.047343,-0.144760,-0.051434,0.993688,-0.190174,-0.111774,-0.021011,-0.023735,-0.11585,0.210284,-0.050860,-0.153472,0.084764,-0.199931,0.347754,-0.052883,0.588244,-0.133966,0.801754,0.786423,-0.334166,-0.080476,-0.056884,-0.029356,0.158972,0.161517,0.0,0.0,0.0
3,-0.195688,-0.031261,-0.053791,-0.024213,0.202181,-0.049132,-0.047343,0.060676,-0.033997,-1.406699,1.097021,-0.207331,-0.022675,-0.081354,-0.11585,-1.332257,-0.042018,-0.153472,-0.006427,-0.199931,-0.663304,-0.004458,0.588244,-0.133966,-0.086726,-0.143996,0.062141,-0.645182,-0.099366,-0.082324,-0.260895,-0.262014,0.0,0.0,0.0
4,-0.195688,-0.031261,-0.053791,-0.023598,-0.209637,-0.076920,-0.047343,0.060676,-0.025278,-0.373805,-0.190174,-0.195596,-0.022675,-0.081354,-0.11585,-0.935603,-0.046439,-0.153472,0.203525,0.212392,-0.073520,-0.018294,0.588244,0.288700,-0.086726,-0.021572,4.704604,-0.645182,-0.071045,-0.055840,-0.120939,-0.120837,0.0,0.0,0.0


In [21]:
import pickle
with open('./Preprocessed_data/v4/testdata.txt', 'wb') as f:
    pickle.dump(df, f)